# ArcGIS Online Web Map Service Validation Audit

Description: This script will check the urls of map services within webmaps to determine if they are valid. 
If they are not valid, it will write the results to a csv file so they can be taken care of. 
 
Created on: 9/3/2018
 
Purpose: This script will go through an entire ArcGIS Online Organization or a Portal Organization and look through all of the Web Maps. Then, this script will check the all of the urls of all of the map services within each Web Map to determine if they are valid. If they are not valid, it will write the results to a csv file so they can be taken care of. The csv file can then be used to aid the administrator in the cleanup of the map services with invalid urls.  This is a Jupyter Notebook written using the ArcGIS Python API. 
 
Authored by: Joe Guzi
 
Previous Production Date: 8/7/2024      

Production Date: 3/11/2025

Note:
 - 8/7/2024 - Updated with my template
 - 3/11/2025 - Script was updated due to the update to the ArcGIS Python API version 2.4, using the web map's JSON to find the layer URL since Map module is not working
 - 4/15/2025 - removed unnecessary module from import block.

# Step 1: Import Libraries

In [ ]:
# import
print("Import Libraries...")
from arcgis.gis import GIS
from arcgis.gis import ItemProperties
from arcgis.gis import ItemTypeEnum
from arcgis.features import FeatureLayerCollection
import getpass
from IPython.display import display
import os, tempfile
import time
import requests
from datetime import date
import json
import requests
import csv
from tkinter.filedialog import asksaveasfilename
print("Import Libraries Complete!")

# Step 2: Connect to ArcGIS Online or ArcGIS Enterprise

Use one of the methods below to conenct to ArcGIS Online or ArcGIS Enterprise.

## Connect with current AGO Credentials to this organization

Use this option if you are already signed into AGO and are running this notebook as a hosted notebook in AGO.

In [ ]:
gis = GIS("home")
gis
print("Connect to ArcGIS Online Complete!")

## Connect to ArcGIS Online by providing Org URL and Credentials (built-in users)

In [ ]:
# Connection Variables
Organization = input("What's your oranization? ")
User = input("What's your  username? ")
Password = getpass.getpass("What's your password? ")

# Connection 
gis = GIS(Organization, User, Password)
gis
print("Connected to AGO")

## Connect to ArcGIS Online by providing a profile

We connect by using a profile. This profile has been setup beforehand by using built-in credentials and registering it with the profile itself. We have a section in the documentation that shows you how to do this if you are interested.

In [ ]:
# Connection Variables
gis = GIS(profile="your_online_profile") # edit "your_online_profile" with the name of your profile
gis
print("Connected to AGO")

## User authentication with OAuth 2.0

https://developers.arcgis.com/python/guide/working-with-different-authentication-schemes/#user-authentication-with-oauth-20

In [ ]:
# Connection Variables
Organization = input("What's your organization? ")
OrgClientID = input("What's your  ClientID? ")
gis = GIS(Organization, client_id=OrgClientID)
print("Successfully logged in as: " + gis.properties.user.username)
print("Connected to AGO")

## Connect to ArcGIS Enterprise with Single Sign On

In [ ]:
# Connection Variables
Organization = input("What's your oranization? ")
gis = GIS(Organization)
gis
print("Logged in as " + gis.properties.user.username + " to " + gis.properties.portalName)
print("Connected to AGE")

# Step 3: Define Functions (Optional)

In [ ]:
print("Begin Function Section.")
# This is where you define Functions

print("Function Section Complete!")

# Step 4: Execute Code Section

In [ ]:
User = gis.users.me.username
#print(User)
OwnerQuery = "owner:" + User
#print(OwnerQuery)
Audit = gis.content.search(query=OwnerQuery, item_type="Web Map", max_items = 10000)
#Audit = gis.content.search(query="id:InsertItemIdForTesting", item_type="Web Map", max_items = 10000) # Item Id of a Web Map
s = ""
for Item in Audit:
    ItemID = str(Item["id"])
    #print("itemid")
    print(ItemID)
    ItemData = Item.get_data(try_json=True)
    #print(ItemData)
    for ItemInfo in ItemData:
        #print("In Item JSON Loop")
        if ItemInfo == "operationalLayers":
            #print("In Operational Layer Loop")
            layers = ItemData["operationalLayers"]
            #print("Define Layers")
            for layer in layers:
                #print("In Layer Loop")
                #print (layer)
                URL = str(layer.get('url'))
                #print(URL)
                if URL != None:
                    try:
                        request = requests.get(URL)
                        if request.status_code == 200:
                            print('Web site exists')
                        else:
                            ItemID = str(Item.get("id"))
                            ItemName = str(Item.get("name"))
                            ItemTitle = str(Item.get("title"))
                            ItemType = str(Item.get("type"))
                            ItemOwner = str(Item.get("owner"))
                            Layername = layer.get("title")
                            URLStatusCode = str(request.status_code)
                            URL = str(layer.get('url'))
                            s += ItemID + "," 
                            ItemTitle = ItemTitle.replace(",", ";")
                            s += ItemTitle.replace('℠', "") + ","
                            s += ItemType + ","
                            s += ItemOwner + ","
                            s += Layername + ","
                            s += URLStatusCode + ","
                            s += URL + ","
                            s += "\n"
                    except:
                        print('Web site does not exists')
                        ItemID = str(Item.get("id"))
                        ItemName = str(Item.get("name"))
                        ItemTitle = str(Item.get("title"))#.replace(',', ""))
                        ItemType = str(Item.get("type"))
                        ItemOwner = str(Item.get("owner"))
                        Layername = layer.get("title")
                        URL = str(layer.get('url'))
                        s += ItemID + "," 
                        ItemTitle = ItemTitle.replace(",", ";")
                        s += ItemTitle.replace('℠', "") + ","
                        s += ItemType + ","
                        s += ItemOwner + ","
                        s += Layername + ","
                        s += "Error" + ","
                        s += URL + ","
                        s += "\n"
                        
print ("Audit Section Complete!")

# Step 5: Write the Results to Excel (Optional)

In [ ]:
print("Begin Write Results to Excel Section.")
filepath = asksaveasfilename() # show an "Open" dialog box and return the path to the selected file
print(filepath)
csvfile = filepath + ".csv"
print(csvfile)
with open(csvfile, 'w') as output:
    print ("Wrtie data To CSV File")
    output.write("WebMapId,WebMapTitle,Type,WebMapOwner,LayerName,LayerStatus,LayerUrl\n")
    output.writelines(s)
    print("Wrtie data To CSV File Complete!")

# Step 6: Upload to AGO (Optional)

In [ ]:
print("Begin Upload to AGO Section.") 
file = filepath.rpartition('/')
#print(file)
filename = file[2]
#print(file[2])
testfile = gis.content.search(query="title:"+filename, item_type="Feature Layer Collection")
if testfile:
    print("Found the service")
    testfile = testfile[0]
    display(testfile)
    from arcgis.features import FeatureLayerCollection
    testfile_collection = FeatureLayerCollection.fromitem(testfile)
    print("The table already exists")
    testfile_collection.manager.overwrite(csvfile)
    display
else:
    print("We have to upload it")
    print("Define User")
    User = gis.users.me.username
    Folder = input("What folder would you like to create your item?")
    # Check if Folder Exists
    FolderExists = gis.content.folders.get(folder=Folder, owner=User)
    print(FolderExists)
    if FolderExists == None:
        print("Create Folder")
        gis.content.folders.create(Folder)
    ItemFolder = gis.content.folders.get(folder=Folder, owner=User)
    print("Define Item Properties")
    item_props = ItemProperties(title=filename + "_csv", item_type=ItemTypeEnum.CSV.value)
    print("Add csv to AGO")
    csv_future_item = ItemFolder.add(item_properties=item_props, file=csvfile)
    csv_item = csv_future_item.result()
    display(csv_item)
    print("Publish csv as Table Layer")
    csv_lyr = csv_item.publish()
    display(csv_lyr)
    
print ("The CSV has been published.\nAll done, thanks for playing!")